# FCC Application Scraping

## Set up selenium webdriver with Firefox
* Download Firefox driver [geckodriver](https://github.com/mozilla/geckodriver/releases) for selenium
* Move the driver executable into the same directory as this notebook (or update the path to the executable in `scrape_table_for_call_sign`)
* `pip install selenium`

In [1]:
from bs4 import BeautifulSoup
import time

In [39]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options  
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException        

## Scraping FCC tables for an individual call sign

In [40]:
def element_exists(driver, xpath):
    """
    Helper function that checks if the element at the given xpath exists on
    driver's current page. This is used to check if there is a "Next" button
    -- to scrape through multiple pages of the FCC's paginated tables.
    """
    try:
        driver.find_element_by_xpath(xpath)
    except NoSuchElementException:
        return False
    return True


In [41]:
def scrape_table_page(innerHTML):
    """
    Extracts the following details from a call sign table page:
        * File Number
        * Call Sign/Lease ID
        * Applicant Name
        * FRN
        * Purpose - split into Purpose_Main and Purpose_Note
        * Radio
        * Service
        * Receipt
        * Date
        * Status
    Returns as a list of lists.
    """
    soup = BeautifulSoup(innerHTML, "lxml")
    table_page = soup.find("table", {"summary": "Application search results"})
    
    # run through each row
    rows = table_page.find_all("tr")[1:-1]
    table_data = []
    for row in rows:  # note: skip the first (header) and last (pagination) rows
        
        # run through each cell of each row
        cols = row.find_all("td")
        row_data = []
        for i in range(1, len(cols)): # note: skip the first (index) cell
            
            # extract cell data, trim leading/trailing whitespace
            cell_data = cols[i].get_text().strip()
            
            # fifth-index cells "Purpose" has to get split into two cols
            if i == 5:
                
                purpose = cell_data.split("\n")
                purpose_main = purpose[0].strip()
                purpose_note = ""
                
                # sometimes the "Purpose" column has an optional note
                if len(purpose) >= 2:
                    purpose_note = purpose[-1].strip()
                    
                row_data.append(purpose_main)
                row_data.append(purpose_note)
            
            # other cells can be inserted as normal
            else:
                row_data.append(cell_data)
                
        table_data.append(row_data)
        
    return(table_data)


In [50]:
def scrape_table_for_call_sign(call_sign, without_browser=False):
    """
    Scrapes the FCC Application Search database for a given call sign.
    
    Returns the table as a list of lists, for example:
    [
      ['File Number', 'Call Sign/Lease ID', 'Applicant Name', 'FRN', 'Purpose_Main', 'Purpose_Note', 'Radio Service', 'Receipt Date', 'Status'],
      ['0000214527',  'KNKN555',            'AT&T',           '',    'Amendment',    'Transfer Control', 'AL',        '09/05/2000',   'Granted'],
      ...,
      ...
    ]
    
    """
    tick = time.time()
    print("LOG[INFO]: START %s " % call_sign)
    
    # driver config - start in private
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
    
    # (optional) don't open up browser
    firefox_options = Options()
    if without_browser:
        firefox_options.add_argument("--headless")
    
    # spin up the webdriver
    firefox = webdriver.Firefox(options=firefox_options, executable_path="./geckodriver")
    firefox.get("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp")
    
    # specify the call sign that we want to scrape for
    callsign = firefox.find_element_by_id("ulsCallSign")
    callsign.send_keys(call_sign)
    
    # set pagination to 100 (maximum)
    pagination = Select(firefox.find_element_by_xpath("//select[@name='pageSize']"))
    pagination.select_by_visible_text("100")
    
    # submit the search form, wait for load
    firefox.find_element_by_name("search").submit()
    WebDriverWait(firefox, 15).until(EC.url_changes("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp"))
    WebDriverWait(firefox, 15).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
    
    table_data = []
    
    # scrape the table on the first page
    innerHTML = firefox.execute_script("return document.body.innerHTML")
    table_data += scrape_table_page(innerHTML)
    
    # as long there is a link to "next page"...
    next_page_xpath = "//*[@title='Next page of results']"
    while element_exists(firefox, next_page_xpath):
        # ... go to next page
        firefox.find_element_by_xpath(next_page_xpath).click()
        # ... wait for load
        WebDriverWait(firefox, 15).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
        # .. and scrape
        innerHTML = firefox.execute_script("return document.body.innerHTML")
        table_data += scrape_table_page(innerHTML)

    # gracefully close the webdriver
    firefox.quit()
    
    tock = time.time()
    print("LOG[INFO]: END %s (%2f seconds)" % (call_sign, tock - tick))
    
    return table_data
     

## Scrape the FCC tables for a list of call signs

In [51]:
# read in the list of FCC callsigns
call_signs = ["KNKN555", "KNKN555"]

In [52]:
# collects the scraped data for *all* the call signs into a single python list
scraped_data = []
for cs in call_signs:
    scraped_data += scrape_table_for_call_sign("KNKN555", without_browser=True)

LOG[INFO]: START KNKN555 
LOG[INFO]: END KNKN555 (12.824816 seconds)
LOG[INFO]: START KNKN555 
LOG[INFO]: END KNKN555 (12.443865 seconds)


In [53]:
# sanity check over the data for KNKN555
assert(len(scraped_data) == 168)
for row in scraped_data:
    assert(len(row) == 9)

In [54]:
# print first 10 rows
for row in scraped_data[:10]:
    print(row)

['0000083337', '', 'AT&T Wireless Services of Florida, Inc.', '', 'Assignment of Authorization', '', 'AL', '02/18/2000', 'Consummated']
['0000214527', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Renewal/Modification', '', 'CL', '09/05/2000', 'Granted']
['0000287045', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Duplicate License', '', 'CL', '12/08/2000', 'Granted']
['0000326671', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Duplicate License', '', 'CL', '01/16/2001', 'Granted']
['0000446751', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Administrative Update', '', 'CL', '05/02/2001', 'Granted']
['0000545823', '', 'AT&T Wireless Services, Inc.', '0004122032', 'Transfer of Control', '', 'AL', '08/01/2001', 'Inactive']
['0000545823', '', 'AT&T Wireless Services, Inc.', '0004122032', 'Amendment', '(Transfer of Control)', 'AL', '12/20/2001', 'Granted']
['0000949139', 'KNKN555', 'AT&T Wireless Services of Florida, Inc.', '', 'Modificat

# - Ignore below this line -

This is code that's probably worth keeping as as reference for now -- in case it gives us insight into solving any bugs we run into. But you can ignore it when you run the scraper.

In [ ]:
def scrape_table_for_call_sign(call_sign):
    
    # driver config - start in private
    firefox_profile = webdriver.FirefoxProfile()
    firefox_profile.set_preference("browser.privatebrowsing.autostart", True)
    
    firefox = webdriver.Firefox(executable_path="./geckodriver")
    firefox.get("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp")
    
    # fill in the text input
    callsign = firefox.find_element_by_id("ulsCallSign")
    callsign.send_keys(call_sign)
    
    # set pagination to 100 (maximum)
    pagination = Select(firefox.find_element_by_xpath("//select[@name='pageSize']"))
    pagination.select_by_visible_text("10")
    
    # submit the search form, wait for load
    firefox.find_element_by_name("search").submit()
    WebDriverWait(firefox, 15).until(EC.url_changes("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp"))
    WebDriverWait(firefox, 15).until(lambda _: firefox.execute_script("return document.readyState") == "complete")
    
    #print(firefox.page_source.encode("utf-8"))
    
    #innerHTML = firefox.execute_script("return document.body.innerHTML")
    #print(innerHTML)
    
    # find the link the next page if it exists
    firefox.find_element_by_xpath("//*[@title='Next page of results']").click()

    #next_link = firefox.find_element_by_xpath("//*[@title=\"Next page of results\"][0]")
    #next_link = firefox.find_element_by_xpath("//a[@title='Next page of results']")
    #print(next_link)
    
    #page_links = firefox.find_element_by_xpath("//a[@href]")
    #print(page_links)
    
    # count the number of pages we need to iterate through
    #pages = firefox.find_element_by_xpath("//*[contains(@title, 'Page')]")
    #pages = firefox.find_element_by_css_selector("//[title*='Page']")
    #print(pages)
    
    # gracefully close the webdriver
    time.sleep(10)
    firefox.quit()

In [ ]:
scrape_table_for_call_sign("KNKN555")

In [ ]:
firefox = webdriver.Firefox(executable_path="./geckodriver")
firefox.get("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp")

# fill in the text input
callsign = firefox.find_element_by_id("ulsCallSign")
callsign.send_keys("KNKN555")

# fill in the dropdown
#dropdown= firefox.find_element_by_id("radioServiceCode")
#dropdown.send_keys("CL - Cellular")

# pagination to max
display = firefox.find_element_by_id("ulsRowsPerPage")
display.send_keys(100)

# submit form
firefox.find_element_by_name("search").submit()
WebDriverWait(firefox, 15).until(EC.url_changes("https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAdvanced.jsp"))
#test = WebDriverWait(firefox, 3).until(EC.staleness_of((By.ID, 'ulsCallSign')))
#test = firefox.implicitly_wait(10);

print(firefox.execute_script("return document.readyState"))

WebDriverWait(firefox, 15).until( lambda _: firefox.execute_script("return document.readyState") == "complete")
print(firefox.current_url)

# scrape out the table
#print(firefox.page_source.encode("utf-8"))
innerHTML = firefox.execute_script("return document.body.innerHTML")
print(innerHTML)

time.sleep(30)
firefox.quit()

In [ ]:
ffx = Firefox(executable_path="./geckodriver")
request_data = {
    "basicSearchType": "basicSearchCallSign",
    "basicSearchTerm": "KNKN555",
    "x": "24",
    "y": "4",
    "exactMatchInd": "Y",
    "jsValidated" : "true",
    "currentPage": "searchAppl.jsp",
    "actionElement": "actionApplicationSearch"
}
ffx.request("POST",
    "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/ApplSearchController",
    data=request_data)

In [ ]:
options = Options()  
#options.add_argument("--headless")  
chrome = webdriver.Chrome("./chromedriver")
chrome.get("www.google.com.au")
#chrome.request("GET", "https://www.google.com.au")
#chrome.request("GET", "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAppl.jsp")

In [ ]:
referrer = "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/searchAppl.jsp"
headers = { "referrer" : referrer }
payload = {
    "basicSearchType": "basicSearchCallSign",
    "basicSearchTerm": "KNKN555",
    "x": 24,
    "y": 4,
    "exactMatchInd": "Y",
    "jsValidated" : "true",
    "currentPage": "searchAppl.jsp",
    "actionElement": "actionApplicationSearch"
}

res = requests.post(
    "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/ApplSearchController",
    data=payload,
    headers=headers,
)

In [ ]:
res.history[1].url

In [ ]:
options = Options()  
#options.add_argument("--headless")  
chrome = Chrome("./chromedriver", options=options)
chrome.request("GET", "https://www.fcc.gov/")

In [ ]:
chrome.request("GET", "https://www.fcc.gov/")

In [ ]:
request_data = {
    "basicSearchType": "basicSearchCallSign",
    "basicSearchTerm": "KNKN555",
    "x": 24,
    "y": 4,
    "exactMatchInd": "Y",
    "jsValidated" : "true",
    "currentPage": "searchAppl.jsp",
    "actionElement": "actionApplicationSearch"
}

res = chrome.request(
    "POST",
    "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/ApplSearchController",
    data=request_data
)

In [ ]:
chrome = Chrome("./chromedriver")
response = chrome.request(
    "POST",
    "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/ApplSearchController",
    data={
        "basicSearchType": "basicSearchCallSign",
        "basicSearchTerm": "KNKN555",
        "x": 24,
        "y": 4,
        "exactMatchInd": "Y",
        "jsValidated" : "true",
        "currentPage": "searchAppl.jsp",
        "actionElement": "actionApplicationSearch"
    }
)

In [ ]:
response = chrome.request(
    "GET",
    "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/results.jsp?applSearchKey=applSearchKey2019321143892"
)

In [ ]:
res = requests.post(
    "https://wireless2.fcc.gov/UlsApp/ApplicationSearch/ApplSearchController",
    data={
        "basicSearchType": "basicSearchCallSign",
        "basicSearchTerm": "KNKN555",
        "x": "24",
        "y": "4",
        "exactMatchInd": "Y",
        "jsValidated" : "true",
        "currentPage": "searchAppl.jsp",
        "actionElement": "actionApplicationSearch"
    }
)